In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190504.txt')
df2 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190511.txt')
df = pd.concat([df1,df2])
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,00:00:00,REGULAR,7035249,2384833
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020


In [15]:
#Convert Date and Time into a more useable DateTime format.
def df_DateTime(df):
    df['DateTime'] = pd.to_datetime(df.DATE+' '+df.TIME)
    return df

#Record what day of the week each entry is from
def df_day_of_week(df):
    df['Day']=df.DateTime.dt.day_name()
    return df

#SCP represents a specific turnstile.
#C/A represents a control area. This is a bank of turnstiles.  
#Unit represents a remote unit. Usually a whole station, or an area in a complex station.
#We can combine all three to create a unique ID for any turnstile
def df_Unit_ID(df):
    df['Unit_ID'] = df.SCP+' '+df['C/A']+' '+df.UNIT
    return df

#Rename Exits column to remove extra spaces
def rename_columns(df):
    df.rename(columns={'EXITS                                                               ':'EXITS'},inplace=True)
    return df

#Remove irregularities in reporting
def clean_desc(df):
    df = df[df.DESC == 'REGULAR']
    return df

#Calculate the entries/exits in a four hour period by finding the difference between rows
def df_four_hour(df):
    df['Four Hour Entries'] = df.groupby('Unit_ID').ENTRIES.diff()
    df['Four Hour Exits'] = df.groupby('Unit_ID').EXITS.diff()
    #Can we make this so that the differences are only taken when the Unit_IDs match?
    return df

#Replace NaN, negative values, and impossibly large values by the average of the turnstile
def df_clean_four_hour(df):
    df['Four Hour Entries'].fillna(-1, inplace=True)
    df['Four Hour Exits'].fillna(-1, inplace=True)
    df['Four Hour Entries'] = df.groupby(['TIME','C/A'])['Four Hour Entries'].transform(
        lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))
    df['Four Hour Exits'] = df.groupby(['TIME','C/A'])['Four Hour Exits'].transform(
        lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))
    return df

#Create a column for the total amount of traffic through a turnstile
def df_total_flow(df):
    df['Total Flow'] = df['Four Hour Entries']+df['Four Hour Exits']
    return df

In [16]:
df1 = df_DateTime(df1)
df1 = df_day_of_week(df1)
df1 = df_Unit_ID(df1)
df1 = rename_columns(df1)
df1 = df_four_hour(df1)
df1 = df_clean_four_hour(df1)
df1 = clean_desc(df1)
df1 = df_total_flow(df1)
df1.head()

<ipython-input-15-91586f5ad53e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DateTime'] = pd.to_datetime(df.DATE+' '+df.TIME)
<ipython-input-15-91586f5ad53e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Day']=df.DateTime.dt.day_name()
<ipython-input-15-91586f5ad53e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DateTime,Day,Unit_ID,Four Hour Entries,Four Hour Exits,Total Flow
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,00:00:00,REGULAR,7035249,2384833,2019-04-27 00:00:00,Saturday,02-00-00 A002 R051,149.152778,49.111111,198.263889
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840,2019-04-27 04:00:00,Saturday,02-00-00 A002 R051,20.000000,7.000000,27.000000
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875,2019-04-27 08:00:00,Saturday,02-00-00 A002 R051,23.000000,35.000000,58.000000
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951,2019-04-27 12:00:00,Saturday,02-00-00 A002 R051,100.000000,76.000000,176.000000
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020,2019-04-27 16:00:00,Saturday,02-00-00 A002 R051,259.000000,69.000000,328.000000


In [18]:
#Find daily sum for each STATION for each day of the week
df1_daily = df1.groupby(['STATION','DATE'])['Four Hour Entries','Four Hour Exits','Total Flow'].sum()
df1_daily.head(20)

<ipython-input-18-699f5f3e626f>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1_daily = df1.groupby(['STATION','DATE'])['Four Hour Entries','Four Hour Exits','Total Flow'].sum()


Four Hour Entries  Four Hour Exits  Total Flow
STATION       DATE                                                      
1 AV          04/27/2019        6611.333333      9176.666667     15788.0
              04/28/2019        5205.000000      7239.000000     12444.0
              04/29/2019       17134.000000     19320.000000     36454.0
              04/30/2019       18588.000000     20421.000000     39009.0
              05/01/2019       19221.000000     21124.000000     40345.0
              05/02/2019       19406.000000     21335.000000     40741.0
              05/03/2019       19732.000000     22665.000000     42397.0
103 ST        04/27/2019       19351.000000     13987.000000     33338.0
              04/28/2019       15913.000000     12349.000000     28262.0
              04/29/2019       30243.000000     21166.000000     51409.0
              04/30/2019       32303.000000     21440.000000     53743.0
              05/01/2019       32706.000000     22290.000000     54996.0
              05/02/2019       32849.000000     22354.000000     55203.0
              05/03/2019       32740.000000     21646.000000     54386.0
103 ST-CORONA 04/27/2019       14378.000000     11945.500000     26323.5
              04/28/2019       11415.000000     10283.000000     21698.0
              04/29/2019       19903.000000     12769.000000     32672.0
              04/30/2019       19854.000000     14780.000000     34634.0
              05/01/2019       20265.000000     13799.000000     34064.0
              05/02/2019       20693.000000     14936.000000     35629.0

In [19]:
df1_daily.sort_values(by=['Total Flow'],ascending=False).head(20)

Four Hour Entries  Four Hour Exits     Total Flow
STATION         DATE                                                         
34 ST-PENN STA  05/02/2019      177841.476190    160361.410714  338202.886905
                05/01/2019      177733.476190    159379.410714  337112.886905
                05/03/2019      169801.476190    158841.410714  328642.886905
                04/30/2019      174815.476190    153589.410714  328404.886905
                04/29/2019      169283.913690    146943.098214  316227.011905
GRD CNTRL-42 ST 05/01/2019      165304.071340    141747.924143  307051.995483
                05/02/2019      163527.355124    142631.302521  306158.657645
                04/30/2019      162523.355124    139958.302521  302481.657645
                05/03/2019      156925.355124    136269.949580  293195.304704
                04/29/2019      150529.258633    132881.184874  283410.443507
34 ST-HERALD SQ 05/03/2019      125888.987302    117832.000000  243720.987302
                05/02/2019      124792.653968    118410.000000  243202.653968
                05/01/2019      124479.987302    116378.000000  240857.987302
                04/30/2019      121956.987302    112160.000000  234116.987302
                04/29/2019      113923.765079    106299.000000  220222.765079
14 ST-UNION SQ  05/02/2019      116460.000000    101153.000000  217613.000000
                05/03/2019      109138.000000    101029.000000  210167.000000
TIMES SQ-42 ST  05/01/2019      108617.680272     97424.000000  206041.680272
14 ST-UNION SQ  04/30/2019      105320.000000     99042.000000  204362.000000
TIMES SQ-42 ST  05/02/2019      106554.680272     97725.000000  204279.680272

In [ ]:
df2 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190511.txt')
df3 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190518.txt')
df4 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190525.txt')
df5 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190601.txt')